In [1]:
from pymongo import MongoClient
c = MongoClient('localhost', 27017, directConnection=True)

In [2]:
# Налаштувати реплікацію в конфігурації: Primary with Two Secondary Members (всі ноди можуть бути запущені як окремі процеси або у Docker контейнерах)

config = {'_id': 'foo', 'members': [
    {'_id': 0, 'host': 'localhost:27017'},
    {'_id': 1, 'host': 'localhost:27018'},
    {'_id': 2, 'host': 'localhost:27019'}]}
c.admin.command("replSetInitiate", config)

{'ok': 1.0,
 'operationTime': Timestamp(1684089769, 1),
 '$clusterTime': {'clusterTime': Timestamp(1684089769, 1),
  'signature': {'hash': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00',
   'keyId': 0}}}

In [7]:
from time import sleep
c = MongoClient(replicaset='foo'); print(c.nodes); sleep(0.1); print(c.nodes)

frozenset()
frozenset({('localhost', 27017), ('localhost', 27018), ('localhost', 27019)})


In [17]:
mydb = c["mydb"]
mytable = mydb["demo"]


In [19]:
# Продемонструвати запис даних на primary node з різними Write Concern Levels (http://docs.mongodb.org/manual/core/write-concern/)
# Unacknowledged

from pymongo.write_concern import WriteConcern
mytable.with_options(write_concern=WriteConcern(w=0)).insert_one({"name": "Oleksii", "email": "example@gmail.com"})

In [20]:
print(c.list_database_names())

['admin', 'config', 'local', 'mydb']


In [22]:
# Acknowledged
mytable.with_options(write_concern=WriteConcern(w=1)).insert_one({"name": "Vadym", "email": "example@gmail.com"})
# Journaled
mytable.with_options(write_concern=WriteConcern(w=1,j=True)).insert_one({"name": "Kostya", "email": "example@gmail.com"})
# AcknowledgedReplica 
mytable.with_options(write_concern=WriteConcern(w="majority")).insert_one({"name": "Nadiia", "email": "example@gmail.com"})

In [34]:
# Продемонструвати Read Preference Modes: читання з primary і secondary node (http://docs.mongodb.org/manual/core/read-preference/)
# Primary

from pymongo import ReadPreference

mytable = MongoClient(read_preference=ReadPreference.PRIMARY, replicaSet='foo')['mydb']['demo']

for post in mytable.find({}):
    print(post)

{'_id': ObjectId('6461313a90b0f670ed25e07f'), 'name': 'Oleksii', 'email': 'example@gmail.com'}
{'_id': ObjectId('6461332d90b0f670ed25e080'), 'name': 'Vadym', 'email': 'example@gmail.com'}
{'_id': ObjectId('6461332d90b0f670ed25e081'), 'name': 'Kostya', 'email': 'example@gmail.com'}
{'_id': ObjectId('6461332d90b0f670ed25e082'), 'name': 'Nadiia', 'email': 'example@gmail.com'}


In [35]:
# Secondary

mytable = MongoClient(read_preference=ReadPreference.SECONDARY, replicaSet='foo')['mydb']['demo']

for post in mytable.find({}):
    print(post)

{'_id': ObjectId('6461313a90b0f670ed25e07f'), 'name': 'Oleksii', 'email': 'example@gmail.com'}
{'_id': ObjectId('6461332d90b0f670ed25e080'), 'name': 'Vadym', 'email': 'example@gmail.com'}
{'_id': ObjectId('6461332d90b0f670ed25e081'), 'name': 'Kostya', 'email': 'example@gmail.com'}
{'_id': ObjectId('6461332d90b0f670ed25e082'), 'name': 'Nadiia', 'email': 'example@gmail.com'}


In [36]:
# Спробувати зробити запис з однією відключеною нодою та write concern рівнім 3 та нескінченім таймаутом. Спробувати під час таймаута включити відключену ноду 

mytable = MongoClient(replicaSet='foo')['mydb']['demo']

mytable.with_options(write_concern=WriteConcern(w="majority")).insert_one({"name": "Volodymyr", "email": "example@gmail.com"})

In [37]:
for post in mytable.find({}):
    print(post)

{'_id': ObjectId('6461313a90b0f670ed25e07f'), 'name': 'Oleksii', 'email': 'example@gmail.com'}
{'_id': ObjectId('6461332d90b0f670ed25e080'), 'name': 'Vadym', 'email': 'example@gmail.com'}
{'_id': ObjectId('6461332d90b0f670ed25e081'), 'name': 'Kostya', 'email': 'example@gmail.com'}
{'_id': ObjectId('6461332d90b0f670ed25e082'), 'name': 'Nadiia', 'email': 'example@gmail.com'}
{'_id': ObjectId('646135a490b0f670ed25e08b'), 'name': 'Volodymyr', 'email': 'example@gmail.com'}


In [38]:
# Аналогічно попередньому пункту, але задати скінченний таймаут та дочекатись його закінчення. 
# Перевірити чи данні записались і чи доступні на читання з рівнем readConcern: “majority”

mytable = MongoClient(replicaSet='foo')['mydb']['demo']

mytable.with_options(write_concern=WriteConcern(w="majority", wtimeout=5000)).insert_one({"name": "Nastya", "email": "example@gmail.com"})

In [43]:
# Аналогічно попередньому пункту, але задати скінченний таймаут та дочекатись його закінчення. 
# Перевірити чи данні записались і чи доступні на читання з рівнем readConcern: “majority”

from pymongo.read_concern import ReadConcern

mytable.with_options(read_concern=ReadConcern("majority")).find_one({'name': 'Nastya'})

{'_id': ObjectId('6461374a90b0f670ed25e08d'),
 'name': 'Nastya',
 'email': 'example@gmail.com'}

In [47]:
mytable = MongoClient(replicaSet='foo')['mydb']['demo']

mytable.with_options(read_concern=ReadConcern("majority")).find_one({'name': 'Nastya'})

{'_id': ObjectId('6461374a90b0f670ed25e08d'),
 'name': 'Nastya',
 'email': 'example@gmail.com'}

In [60]:
# Продемонстрував перевибори primary node в відключивши поточний primary (Replica Set Elections) 
# Відключаємо основну ноду і намагаємось підключитись до нього
db = MongoClient("localhost", replicaSet='foo')['mydb']

In [65]:
# Намагаємось знайти будь який запис
db.demo.find_one()

ServerSelectionTimeoutError: localhost:27017: [Errno 111] Connection refused, Timeout: 30s, Topology Description: <TopologyDescription id: 64613aee90b0f670ed25e097, topology_type: ReplicaSetNoPrimary, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 111] Connection refused')>]>

In [71]:
# Підключаємось до іншої ноди
db = MongoClient("localhost", 27018, replicaSet='foo')['mydb']
# Тестове вивантаження даних
db.demo.find_one()

{'_id': ObjectId('6461313a90b0f670ed25e07f'),
 'name': 'Oleksii',
 'email': 'example@gmail.com'}

In [72]:
# Робимо запис і вичитку даних з ноди
db.demo.insert_one({"name": "Marko", "email": "example@gmail.com"})
db.demo.find_one({'name': 'Marko'})

{'_id': ObjectId('64613c4490b0f670ed25e09b'),
 'name': 'Marko',
 'email': 'example@gmail.com'}

In [79]:
# Підключаємось назад до колишньої основної ноди і робимо вичитку запису
db = MongoClient("localhost", 27017, replicaSet='foo')['mydb']
db.demo.find_one({'name': 'Marko'})

{'_id': ObjectId('64613c4490b0f670ed25e09b'),
 'name': 'Marko',
 'email': 'example@gmail.com'}

In [84]:
# Привести кластер до неконсистентного стану користуючись моментом часу коли primary node не відразу помічає відсутність secondary node 
# відключивши дві secondary node протягом 5 сек. на мастер записати значення (з w:1) і перевірити, що воно записалось

# Підключаємось і робимо запис до основної ноди
db = MongoClient("localhost", 27017, replicaSet='foo')['mydb']
db.demo.with_options(write_concern=WriteConcern(w=1)).insert_one({"name": "Senya", "email": "example@gmail.com"})

In [90]:
# спробувати зчитати це значення з різними рівнями read concern - readConcern: {level: <"majority"|"local"| "linearizable">}
# Спробував всі три - випадає еррора з таймаутом

db.demo.with_options(read_concern=ReadConcern("local")).find_one({'name': 'Senya'})

ServerSelectionTimeoutError: No replica set members match selector "Primary()", Timeout: 30s, Topology Description: <TopologyDescription id: 6463ceb190b0f670ed25e0a5, topology_type: ReplicaSetNoPrimary, servers: [<ServerDescription ('localhost', 27017) server_type: RSSecondary, rtt: 0.0003758249327729074>, <ServerDescription ('localhost', 27018) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27018: [Errno 111] Connection refused')>, <ServerDescription ('localhost', 27019) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27019: [Errno 111] Connection refused')>]>

In [114]:
# включити дві інші ноди таким чином, щоб вони не бачили попереднього мастера (його можна відключити) і дочекатись поки вони оберуть нового мастера 
# Підключаємось до репліка сету
c = MongoClient(replicaSet='foo')

In [115]:
# Перевіряємо яка нода основна. Раніше це була 27017
c.primary

('localhost', 27017)

In [116]:
# Намагаємось знайти значення яке записали на колишню основну ноду - безуспішно.
c.db.demo.find_one({'name': 'Senya'})

In [134]:
# Земулювати eventual consistency за допомогою установки затримки реплікації для репліки 
c = MongoClient('localhost', 27017, directConnection=True)

In [135]:
c.admin.command('replSetGetConfig')

{'config': {'_id': 'foo',
  'version': 18098,
  'protocolVersion': 1,
  'members': [{'_id': 0,
    'host': 'localhost:27017',
    'arbiterOnly': False,
    'buildIndexes': True,
    'hidden': False,
    'priority': 1.0,
    'tags': {},
    'slaveDelay': 0,
    'votes': 1},
   {'_id': 1,
    'host': 'localhost:27018',
    'arbiterOnly': False,
    'buildIndexes': True,
    'hidden': True,
    'priority': 0.0,
    'tags': {},
    'slaveDelay': 0,
    'votes': 1},
   {'_id': 2,
    'host': 'localhost:27019',
    'arbiterOnly': False,
    'buildIndexes': True,
    'hidden': False,
    'priority': 1.0,
    'tags': {},
    'slaveDelay': 0,
    'votes': 1}],
  'settings': {'chainingAllowed': True,
   'heartbeatIntervalMillis': 2000,
   'heartbeatTimeoutSecs': 10,
   'electionTimeoutMillis': 10000,
   'catchUpTimeoutMillis': -1,
   'catchUpTakeoverDelayMillis': 30000,
   'getLastErrorModes': {},
   'getLastErrorDefaults': {'w': 1, 'wtimeout': 0},
   'replicaSetId': ObjectId('64612ba91b9304d92f

In [144]:
# Change config for delayed Replica Set Member
config = {
    'version': 18099,
    '_id': 'foo', 'members': [
    {'_id': 0, 'host': 'localhost:27017'},
    {'_id': 1,
     'host': 'localhost:27018',
     'priority': 0,
     'hidden': True,
     'slaveDelay': 120},
    {'_id': 2, 'host': 'localhost:27019'}]}
c.admin.command({'replSetReconfig': config})

{'ok': 1.0,
 'operationTime': Timestamp(1684266274, 1),
 '$clusterTime': {'clusterTime': Timestamp(1684266274, 1),
  'signature': {'hash': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00',
   'keyId': 0}}}

In [147]:
c.mydb.demo.insert_one({"name": "Varnava", "email": "example@gmail.com"})

In [148]:
c.mydb.demo.with_options(read_concern=ReadConcern("linearizable")).find_one({'name': 'Varnava'})

{'_id': ObjectId('6463de5b90b0f670ed25e0bf'),
 'name': 'Varnava',
 'email': 'example@gmail.com'}